## **GF Project 5**

**Project Goal and Objectives**: Have the bot every 30 mins from 9am - 9pm EST give an update on the traffic flow and compare those values with MAs.

### **Bot**:
 - 1) Trigger the file request using a trigger: Trigger + Playwirght 
 - 2) Call the notebook (NEF_Project_5_local.ipynb)
 - 3) Output the print message from the notebook to the chat
 - 4) Keep in the directory the files shown in ORIGINAL_FILES list

### **NEF_Project_%_local.ipynb**:
- 1) Will start by impoting the file (Traffic View 3)
- 2) Calculate all the present stats 
- 3) Insert into the databse the values for the given day and hour 
- 4) Comapre result with MAs from same time of previous last week, if standard deviaiton >20% --> ALERT 

#### STEP 1 - Importing Traffic View for a given instance 

In [1]:
import os 
import pandas as pd

current_dir = os.getcwd()
files = os.listdir(current_dir)
for file in files:
    if "Project5" in file and file.endswith('.csv'):
        print(file)
        file_name = file
        
tview1 = pd.read_csv(file_name)
tview1.head()



Project5.csv
Project5_2025-06-23.csv


,Item,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,Successful Unique Cycle Attempts,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,218,192,88.07%,26,11.93%,204,178,87.25%,26,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$68.30,13.86%,$424.40,86.14%
1,DP1 V3,989,669,67.64%,320,32.36%,601,281,46.76%,320,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$402.60,9.76%,"$3,606.19",87.43%
2,DP1 V3(PP),109,84,77.06%,25,22.94%,109,84,77.06%,25,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$37.80,11.73%,$271.56,84.27%
3,DP12 RP V3,6,6,100.00%,0,0.00%,4,4,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.80,0.00%,$-0.80,0.00%
4,DP12 V3,324,267,82.41%,57,17.59%,252,195,77.38%,57,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$303.82,5.39%,"$5,234.02",92.86%


In [2]:
tview1["Item"].unique()

array(['DP1 RP V3', 'DP1 V3', 'DP1 V3(PP)', 'DP12 RP V3', 'DP12 V3',
       'DP12 V3(PP)', 'DP13 RP V3', 'DP13 V3', 'DP13 V3 (PP)', 'DP2 V3',
       'DP2 V3(PP)', 'DP22 V3', 'DP22 V3(PP)', 'DP23 V3'], dtype=object)

#### **STEP 2 - Calculate the current statistics**

**Guide**:
- S1 Initials is DP1 from Traffic/Scrub/Prepaid
- S1 RP is DP1RP from Traffic/Scrub/Prepaid
- S1LTV and UPLTV are always static

Main difference here is that we dont have the campaign column, so we need to base our dfs solely from the Item column:
- PP: Prepaid
- RP: Reprocess

In [3]:
# We will start off by calculating S1
main_dict = {}
s1_df = tview1[
      (tview1["Item"] == "DP1 V3(PP)") | (tview1["Item"] == "DP1 V3") 
  ]

# Check if column is string/object type before applying string operations
if s1_df["Successful Charges"].dtype == 'object':
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
else:
    s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


total_s1 = s1_df["Successful Charges"].sum()
print(f"Total S1 Successful Charges: {total_s1}")

# Get the Gross
S1G = total_s1 * 9.49

# Get the new S1
S1N = S1G * 0.3

main_dict["S1"] = [total_s1, S1G, S1N]

Total S1 Successful Charges: 345


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\1565210488.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1_df["Successful Charges"] = s1_df["Successful Charges"].astype(int)


In [4]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


982.215
2.847


In [5]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


982.215
2.847


In [6]:
# Now we will calculate S1RP
s1rp_df = tview1[
    (tview1["Item"] == "DP1 RP V3")

]
s1rp_df.head()
s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 26


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\172734612.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [7]:
# Now we will calculate UP

up_df = tview1[
      (tview1["Item"] == "DP2 V3") | (tview1["Item"] == "DP2 V3(PP)")
  ]
up_df.head()
up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

total_up = up_df["Successful Charges"].sum()
print(f"Total UP Successful Charges: {total_up:,}")

# Get the Gross
UPG = total_up * 38.11

# Get the Net
UPN = UPG * 0.23

main_dict["UP"] = [total_up, UPG, UPN]


Total UP Successful Charges: 48


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\3688151986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  up_df["Successful Charges"] = up_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [8]:
# Now we will add the WHC row
whc = total_s1 
whc_g = whc * 3
whc_n = whc_g

# Now we add it to the dic 
main_dict["WHC"] = [whc, whc_g, whc_n]


In [9]:
# NOw we will add UR
ur = total_s1
ur_g = ur * 2
ur_n = ur_g

# Now we add it to the dic 
main_dict["UR"] = [ur, ur_g, ur_n]





In [10]:
# Now ew will add S1 RB Proj
s1rb_proj = (total_s1 + total_s1rp) * 0.2
s1rb_proj_g = s1rb_proj * 115.73
s1rb_proj_n = s1rb_proj_g * 0.25

# Now we add it to the dic 
main_dict["S1RB Proj"] = [s1rb_proj, s1rb_proj_g, s1rb_proj_n]


In [11]:
# Now we will ad UP RB Proj
uprb_proj = total_up * 0.25
uprb_proj_g = uprb_proj * 38.11
uprb_proj_n = uprb_proj_g * 0.5

# Now we add it to the dic 
main_dict["UPRB Proj"] = [uprb_proj, uprb_proj_g, uprb_proj_n]




In [12]:
# Now we will calculate S1RP
s1rp_df = tview1[
    (tview1["Item"] == "DP1 RP V3")
]
s1rp_df.head()

s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)
total_s1rp = s1rp_df["Successful Charges"].sum()
print(f"Total S1RP Successful Charges: {total_s1rp:,}")

# Get the Gross
S1RP_G = total_s1rp * 18.55

# Getthe Net 
S1RP_N = S1RP_G * 0.12

main_dict["S1RP"] = [total_s1rp, S1RP_G, S1RP_N]


Total S1RP Successful Charges: 26


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\4091856167.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1rp_df["Successful Charges"] = s1rp_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [13]:
# Calculate NR/C (pCPA)
net_total = 0
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

NRCPA =  net_total / total_s1
print(NRCPA)



        


5561.276900000001
16.1196431884058


In [14]:
ecpa_df = tview1[
    (tview1["Item"] == "DP1 V3")
]
ecpa_df.head()

ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)

dp1_traffic = ecpa_df["Successful Charges"].sum()

ecpa = round((dp1_traffic / ((total_s1 + total_s1rp))) * -30, 2)
print(ecpa)


-25.88


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\620824186.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ecpa_df["Successful Charges"] = ecpa_df["Successful Charges"].astype(str).str.replace(',', '').astype(int)


In [15]:
S1LTV = 3.50
UPLTV = 1.5




gross_total = 0
for key, values in main_dict.items():
    gross_total += values[1]
print(gross_total)

middle = main_dict["WHC"][1] + main_dict["UR"][1]

RES = (((gross_total - middle)*0.1)/total_s1)+1
print(RES)


NRC = NRCPA + ecpa + S1LTV + UPLTV + RES
print(NRC)







upsell_take = round((total_up / (total_s1))*100, 2)
print(upsell_take)






16355.116000000002
5.240613333333334
0.48025652173913436
13.91


In [16]:
# And in S1ltv
s1ltv = total_s1
s1ltv_g = s1ltv * 4
s1ltv_n = s1ltv_g 

# Add to the dict 
main_dict["S1LTV"] = [s1ltv, s1ltv_g, s1ltv_n]


In [17]:
# And upltv
upltv = total_s1
upltv_g = upltv * 2
upltv_n = upltv_g 

# Add to the dict 
main_dict["UPLTV"] = [upltv, upltv_g, upltv_n]




In [18]:
# Add iin the res 
res = total_s1
res_g = (((((main_dict["S1"][1] + main_dict["S1RP"][1] + main_dict["UP"][1] + main_dict["S1RB Proj"][1] + main_dict["UPRB Proj"][1]) * 0.1) / total_s1) + 3) * total_s1)
res_n = res_g 

# Add to the dict 
main_dict["RES"] = [res, res_g, res_n]




NRC =  ecpa 
print(NRC)


-25.88


In [19]:
# Now we need to add in eCPA
eCPA =  dp1_traffic 
eCPA_g = eCPA * -30
eCPA_n = eCPA_g 

# Add to the dict 
main_dict["eCPA"] = [eCPA, eCPA_g, eCPA_n]  


sum_for_nrc_new = 0
for key, values in main_dict.items():
    if key != "eCPA":
        sum_for_nrc_new += values[2]

nrc_new = sum_for_nrc_new / (total_s1 + total_s1rp)
nrc_new



27.302664420485176

In [20]:
main_dict

{'S1': [345, 3274.05, 982.215],
 'S1RP': [26, 482.3, 57.876],
 'UP': [48, 1829.28, 420.7344],
 'WHC': [345, 1035, 1035],
 'UR': [345, 690, 690],
 'S1RB Proj': [74.2, 8587.166000000001, 2146.7915000000003],
 'UPRB Proj': [12.0, 457.32, 228.66],
 'S1LTV': [345, 1380, 1380],
 'UPLTV': [345, 690, 690],
 'RES': [345, 2498.0116000000003, 2498.0116000000003],
 'eCPA': [320, -9600, -9600]}

In [21]:
from datetime import datetime
import pytz

# Get current time in New York
ny_timezone = pytz.timezone('America/New_York')
ny_time = datetime.now(ny_timezone)

# Format as 12-hour with AM/PM
formatted_time = ny_time.strftime('%#I:%M %p') 
print(formatted_time)

6:43 PM


In [22]:
# TV analysis message 
message = f"📺 TV message (NY: {formatted_time}): \n"

# Add teh number of customer, which is total_s1 + total_s1rp
message += f"• Number of customers: {total_s1 + total_s1rp:,}\n"

# Add the upsell take, 
message += f"• Upsell take: {upsell_take:,}%\n"

# Add the ecpa
message += f"• eCPA: {ecpa:,}\n"

# Add the NRC
nrc = nrc_new + ecpa
message += f"• NR/C: {round(nrc, 2):,}\n"


print(message)
list_current = []
list_current.append(total_s1 + total_s1rp)
list_current.append(upsell_take)
list_current.append(ecpa)
list_current.append(nrc)



present_value_customers = (total_s1 + total_s1rp)
present_values_upsell_take = upsell_take
present_value_ecpa = ecpa
present_value_nrc = nrc 

📺 TV message (NY: 6:43 PM): 
• Number of customers: 371
• Upsell take: 13.91%
• eCPA: -25.88
• NR/C: 1.42



### Import the Dataframe 

In [23]:
database_df = pd.read_excel("tv_database_gf.xlsx")

# Create a column where wonly have the date and no time 
database_df["Date"] = database_df["Date-Time"].dt.date

database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP1 V3,DP1 RP V3,total_up
0,2025-06-08 00:30:00,0,0.00,0.00,0.000000,2025-06-08,00:30:00,0,0,0
1,2025-06-08 01:00:00,0,0.00,0.00,0.000000,2025-06-08,01:00:00,0,0,0
2,2025-06-08 01:30:00,0,0.00,0.00,0.000000,2025-06-08,01:30:00,0,0,0
3,2025-06-08 02:00:00,0,0.00,0.00,0.000000,2025-06-08,02:00:00,0,0,0
4,2025-06-08 02:30:00,0,0.00,0.00,0.000000,2025-06-08,02:30:00,0,0,0
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-23 21:00:00,331,8.97,-24.47,2.187261,2025-06-23,21:00:00,1062,174,117
438,2025-06-23 21:00:00,312,9.56,-25.96,0.743553,2025-06-23,21:00:00,1062,174,117
439,2025-06-23 21:00:00,312,9.56,-25.96,0.743553,2025-06-23,21:00:00,1062,174,117
440,2025-06-23 22:00:00,312,9.56,-25.96,0.743553,2025-06-23,22:00:00,1062,174,117


In [24]:
# Get the average of the customers, upsell, ecpa, nrc
real_average = database_df["customers"].mean()
real_average_upsell = database_df["upsell"].mean()
real_average_ecpa = database_df["excpa"].mean()
real_average_nrc = database_df["nrc"].mean()



print(real_average)
print(real_average_upsell)
print(real_average_ecpa)
print(real_average_nrc)





3.7058823529411766
0.11665158371040725
-0.2901131221719457
0.013213994038857327


In [25]:
# Get the rounded hour / half hour interval 
from datetime import datetime, timedelta

def get_current_rounded_time(x):
    """
    Get current time rounded to nearest half hour.
    Returns datetime object rounded to nearest half hour.
    """
    now = x
    
    # Calculate minutes to add/subtract to round to nearest half hour
    minutes = now.minute
    if minutes < 15:
        # Round down to previous half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0)
    elif minutes < 45:
        # Round to current half hour
        rounded_time = now.replace(minute=30, second=0, microsecond=0)
    else:
        # Round up to next half hour
        rounded_time = now.replace(minute=0, second=0, microsecond=0) + timedelta(hours=1)
    
    return rounded_time
timey = datetime.now()
datetimee = get_current_rounded_time(timey)
print(datetimee)
database_df.shape

2025-06-23 23:30:00


(442, 10)

In [26]:
from datetime import datetime
from zoneinfo import ZoneInfo
timeest = timey = datetime.now(ZoneInfo("America/New_York"))
timei = get_current_rounded_time(timeest)
timei.hour, timei.minute
timei = timei.strftime("%H:%M")
timei


'18:30'

### In this section we integartes the Attempts for DP1, DP1 RP, DP 2 etc

In [27]:
tview1

,Item,Attempted Charges,Declined Charges,Declined Charge Rate,Successful Charges,Success Charge Rate,Unique Cycle Attempts,Declined Unique Cycle Attempts,Declined Unique Cycle Attempt Rate,Successful Unique Cycle Attempts,...,Ad Spend,Ad Spend Rate,Chargeback Fees,Chargeback Fees Rate,Alert Fees,Alert Fees Rate,Processing Fees,Processing Fees Rate,Profit,Profit Margin
0,DP1 RP V3,218,192,88.07%,26,11.93%,204,178,87.25%,26,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$68.30,13.86%,$424.40,86.14%
1,DP1 V3,989,669,67.64%,320,32.36%,601,281,46.76%,320,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$402.60,9.76%,"$3,606.19",87.43%
2,DP1 V3(PP),109,84,77.06%,25,22.94%,109,84,77.06%,25,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$37.80,11.73%,$271.56,84.27%
3,DP12 RP V3,6,6,100.00%,0,0.00%,4,4,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.80,0.00%,$-0.80,0.00%
4,DP12 V3,324,267,82.41%,57,17.59%,252,195,77.38%,57,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$303.82,5.39%,"$5,234.02",92.86%
5,DP12 V3(PP),13,11,84.62%,2,15.38%,13,11,84.62%,2,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$12.48,6.31%,$185.30,93.69%
6,DP13 RP V3,14,13,92.86%,1,7.14%,9,8,88.89%,1,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$6.14,6.21%,$92.75,93.79%
7,DP13 V3,66,50,75.76%,16,24.24%,55,39,70.91%,16,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$75.56,4.78%,"$1,506.68",95.22%
8,DP13 V3 (PP),1,1,100.00%,0,0.00%,1,1,100.00%,0,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$0.20,0.00%,$-0.20,0.00%
9,DP2 V3,162,117,72.22%,45,27.78%,162,117,72.22%,45,...,$0.00,0.00%,$0.00,0.00%,$0.00,0.00%,$139.95,6.50%,"$2,012.85",93.50%


In [28]:
### In this section we integrate the Attempts for DP1, DP1 RP, DP2 etc

# DP 1
dddp1_df = tview1[
      (tview1["Item"] == "DP1 V3(PP)") | (tview1["Item"] == "DP1 V3") | (tview1["Item"] == "DP1 RP V3")
  ]

dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(str)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(float)
averagedp1 = dddp1_df["Attempted Charges"].sum()
print(averagedp1)

# DP 1 RP
dddp1rp_df = tview1[tview1["Item"] == 'DP1 RP V3']
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(str)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp1rp_df["Attempted Charges"] = dddp1rp_df["Attempted Charges"].astype(float)
averagedp1rp = dddp1rp_df["Attempted Charges"].sum()
print(averagedp1rp)

# DP 2
dddp2_df = tview1[
      (tview1["Item"] == "DP2 V3") | (tview1["Item"] == "DP2 V3(PP)")
  ]
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(str)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].str.replace(',', '', regex=False)
dddp2_df["Attempted Charges"] = dddp2_df["Attempted Charges"].astype(float)
averagedp2 = dddp2_df["Attempted Charges"].sum()
print(averagedp2)

# Append these to the list_current
list_current.append(averagedp1)
list_current.append(averagedp1rp)
list_current.append(averagedp2)


1316.0
218.0
187.0


C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\515644210.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].astype(str)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\515644210.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dddp1_df["Attempted Charges"] = dddp1_df["Attempted Charges"].str.replace(',', '', regex=False)
C:\Users\mmsou\AppData\Local\Temp\ipykernel_38356\515644210.py:10: SettingWithCopyWarning: 
A value is trying to be se

In [29]:
database_df.columns

Index(['Date-Time', 'customers', 'upsell', 'excpa', 'nrc', 'Date', 'Time',
       'DP1 V3', 'DP1 RP V3', 'total_up'],
      dtype='object')

In [30]:
# Create a new row as a dictionary
new_row = {
    'Date-Time': datetimee,
    'customers': present_value_customers,
    'upsell': present_values_upsell_take,
    'excpa': present_value_ecpa,
    'nrc': present_value_nrc,
    'Date': datetimee.now().date(),
    "Time": datetimee.now().time(),
    "DP1 V3": averagedp1,
    "DP1 RP V3": averagedp1rp,
    "total_up": averagedp2

}

# Add the row using loc
database_df.loc[len(database_df)] = new_row

# Now we drop the oldest one
database_df.drop(index=0, inplace=True)
database_df = database_df.reset_index(drop=True)
database_df["Time"] = database_df["Date-Time"].dt.time


database_df

,Date-Time,customers,upsell,excpa,nrc,Date,Time,DP1 V3,DP1 RP V3,total_up
0,2025-06-08 01:00:00,0,0.00,0.00,0.000000,2025-06-08,01:00:00,0.0,0.0,0.0
1,2025-06-08 01:30:00,0,0.00,0.00,0.000000,2025-06-08,01:30:00,0.0,0.0,0.0
2,2025-06-08 02:00:00,0,0.00,0.00,0.000000,2025-06-08,02:00:00,0.0,0.0,0.0
3,2025-06-08 02:30:00,0,0.00,0.00,0.000000,2025-06-08,02:30:00,0.0,0.0,0.0
4,2025-06-08 03:00:00,0,0.00,0.00,0.000000,2025-06-08,03:00:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
437,2025-06-23 21:00:00,312,9.56,-25.96,0.743553,2025-06-23,21:00:00,1062.0,174.0,117.0
438,2025-06-23 21:00:00,312,9.56,-25.96,0.743553,2025-06-23,21:00:00,1062.0,174.0,117.0
439,2025-06-23 22:00:00,312,9.56,-25.96,0.743553,2025-06-23,22:00:00,1062.0,174.0,117.0
440,2025-06-23 23:30:00,371,13.91,-25.88,1.422664,2025-06-23,23:30:00,1316.0,218.0,187.0


##### Get the average values for each half an hour interval

In [31]:
from datetime import time
unique_times = database_df["Time"].unique().tolist()
for key, values in main_dict.items():
    net_total += values[2]
print(net_total)

# Create the percenatge change function 
def percentage_change(new, old):
    if new != old:
        p = ((new - old)/old)*100
        return p
    if new == 0 or old == 0:
        return "Invalid"
    else:
        return 1

def alert_maker(list_of_mass, list_of_currentt):
    name_of_componets = ["Customer Sales", "Upsell", "Ecpa", "Nrc", "DP1 V3", "DP1 RP V3", "total_up"]
    alerts = {}
    alerts_cons = {}
    if not len(list_current) == len(list_mas):
        return None, None
    
    for i in range(len(list_current)):
        new = list_of_currentt[i]
        old = list_of_mass[i]
        diff = percentage_change(new, old)
        diff = round(diff, 2)

        if abs(diff) > 20:
            alerts[name_of_componets[i]] = [new, diff]
        if abs(diff) == 1:
            alerts_cons[name_of_componets[i]] = [new, diff]

    # Return empty dicts instead of None if no alerts
    if not alerts:
        alerts = {}
    if not alerts_cons:
        alerts_cons = {}
        
    print(alerts_cons)
    exclude_keys = {"total_up", "DP1 RP V3", "DP1 V3"}
    alerts = {k: v for k, v in alerts.items() if k not in exclude_keys}
    return alerts, alerts_cons




print(datetimee.hour, datetimee.minute)
df = database_df[database_df["Time"] == time(datetimee.hour, datetimee.minute)]


# Create a list with all the moving averages
list_mas = []

moving_average_customers = df["customers"].mean()
list_mas.append(moving_average_customers)
moving_average_upsell = df["upsell"].mean()
list_mas.append(moving_average_upsell)
moving_average_ecpa = df["excpa"].mean()
list_mas.append(moving_average_ecpa)
moving_average_ncr = df["nrc"].mean()
list_mas.append(moving_average_ncr)

# Get the previous attemot values for the varipus DPs
df_prev = database_df.iloc[[-2]]
moving_averagedp1 = df_prev["DP1 V3"].values[0]
list_mas.append(moving_averagedp1)

moving_averagedp1rp = df_prev["DP1 RP V3"].values[0]
list_mas.append(moving_averagedp1rp)

moving_averagedp2 = df_prev["total_up"].values[0]
list_mas.append(moving_averagedp2)

print(f"The values for dp1, dp1rp and dp2 are {moving_averagedp1}, {moving_averagedp1rp}, {moving_averagedp2}")
print()


alert_component = alert_maker(list_mas, list_current)
print(f"The current values for the moving averages are: {list_mas}")
print(f"The current values of this analysis are: {list_current}")
print(alert_component[0])
print(alert_component[1])

6090.565400000001
23 30
The values for dp1, dp1rp and dp2 are 1316.0, 218.0, 187.0

{'DP1 V3': [1316.0, 1], 'DP1 RP V3': [218.0, 1], 'total_up': [187.0, 1]}
The current values for the moving averages are: [61.833333333333336, 2.3183333333333334, -4.3133333333333335, 0.23711073674752947, 1316.0, 218.0, 187.0]
The current values of this analysis are: [371, 13.91, -25.88, 1.4226644204851766, 1316.0, 218.0, 187.0]
{'Customer Sales': [371, 500.0], 'Upsell': [13.91, 500.0], 'Ecpa': [-25.88, 500.0], 'Nrc': [1.4226644204851766, 500.0]}
{'DP1 V3': [1316.0, 1], 'DP1 RP V3': [218.0, 1], 'total_up': [187.0, 1]}


In [32]:
print(alert_component[0])

{'Customer Sales': [371, 500.0], 'Upsell': [13.91, 500.0], 'Ecpa': [-25.88, 500.0], 'Nrc': [1.4226644204851766, 500.0]}


In [33]:
message += "\n"
output_message = message
if alert_component[0] or alert_component[1]:
    if alert_component[0]:
        message += "⚠️We have detected alerts: \n"
        for key, value in alert_component[0].items():
            message += f'• {key} has a deviaition of {value[1]}% from its respective moving average at {timei}. \n'
        output_message = message
    if alert_component[1]:
        for key, value in alert_component[1].items():
            output_message += f'• {key} attempts are equal to those half an hour ago. \n'

print(output_message)


📺 TV message (NY: 6:43 PM): 
• Number of customers: 371
• Upsell take: 13.91%
• eCPA: -25.88
• NR/C: 1.42

⚠️We have detected alerts: 
• Customer Sales has a deviaition of 500.0% from its respective moving average at 18:30. 
• Upsell has a deviaition of 500.0% from its respective moving average at 18:30. 
• Ecpa has a deviaition of 500.0% from its respective moving average at 18:30. 
• Nrc has a deviaition of 500.0% from its respective moving average at 18:30. 
• DP1 V3 attempts are equal to those half an hour ago. 
• DP1 RP V3 attempts are equal to those half an hour ago. 
• total_up attempts are equal to those half an hour ago. 



In [34]:
# Delete the file tv_database.xlsx from the directory 
file_path = "tv_database_gf.xlsx"
os.remove(file_path)

# Save the current database_df as tv_database.xlsx
database_df.to_excel("tv_database_gf.xlsx", index=False)